In [1]:
# Import all the things

import json
import pandas as pd
import numpy as np
import wrangle
import acquire
from prepare import basic_clean, tokenize, lemmatize, stem, remove_stopwords, prep_string_data#, split_data

import re
from sklearn.feature_extraction.text import TfidfVectorizer

from wordcloud import WordCloud
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

import nltk

from scipy import stats

import warnings
warnings.filterwarnings("ignore")

In [2]:
# use our own wrangle to get the data
# df = wrangle.get_npr_data()
# df.head()

### I'm interested in topic modeling on a story-level basis
### So I will need to groupby story_id_num

In [3]:
# df = df.sort_values(by=['story_id_num','utterance_order'])
# df.head()

In [4]:
# df = df[['story_id_num','utterance_order','title','lemmatized']]

In [5]:
# df_sample = df[:100]
# df_sample

In [6]:
# df_sample.shape

In [7]:
# pd.DataFrame(df_sample.groupby('title').apply(list).reset_index())

In [8]:
# df_sample.groupby(['story_id_num']).title.first()

In [9]:
# # group every story as an aggregation of all its utterances
# df_sample.groupby(['story_id_num']).lemmatized.agg(' '.join)#.iloc[0]

In [10]:
# # group every story as an aggregation of all its utterances and store in a DF
# stories_test = pd.DataFrame(df_sample.groupby(['story_id_num']).lemmatized.agg(' '.join))
# stories_test

In [11]:
# stories.iloc[0].values

### Run the above on the whole df

In [12]:
# df = df.sort_values(by=['story_id_num','utterance_order'])
# df.head()

In [13]:
# df.dtypes

In [14]:
# df.isna().sum()

In [15]:
# df = df.dropna()

#### well...it worked on the sample above
#### had to drop 2 nulls that existed in the lemmatized column

In [16]:
# # group every story as an aggregation of all its utterances and store in a DF
# stories = pd.DataFrame(df.groupby(['story_id_num']).lemmatized.agg(' '.join))
# stories

-  61,165 stories to work with here

In [17]:
# this attempt to get the word count for every story isn't working currently
# stories['word_count'] = df.lemmatized.apply(str.split).apply(len)
# stories.head()

In [18]:
# sia = nltk.sentiment.SentimentIntensityAnalyzer()
# stories['vader'] = stories.lemmatized.apply(lambda doc: sia.polarity_scores(doc)['compound'])
# stories.head()

In [19]:
# stories.to_csv('stories.csv')

In [20]:
stories = pd.read_csv('stories.csv')
stories.head()

,Unnamed: 0,story_id_num,lemmatized,vader,title
0,0,17,he is in cahoot with a foreign government to m...,0.9964,n.j. democrat on reported whistleblower compla...
1,1,18,"come with concrete plan , not beautiful speech...",0.9993,"ahead of climate summit, 2 views from cities i..."
2,2,19,if you stand outside the old dekalb county cou...,0.9974,"barred from removing confederate monument, cou..."
3,3,20,you may have spotted pink salt in your superma...,0.4950,pakistan's pink himalayan salt has become a ma...
4,4,21,"singing in foreign language . the other day , ...",0.9730,"10 years after haiti's earthquake, 'this music..."


In [21]:
stories = stories.drop(columns = ['Unnamed: 0'])

In [22]:
stories.shape

(61165, 4)

In [23]:
# len(df.groupby(['story_id_num']).title.first())

NameError: name 'df' is not defined

In [24]:
# stories['title'] = df.groupby(['story_id_num']).title.first().tolist()

In [25]:
stories.head()

,story_id_num,lemmatized,vader,title
0,17,he is in cahoot with a foreign government to m...,0.9964,n.j. democrat on reported whistleblower compla...
1,18,"come with concrete plan , not beautiful speech...",0.9993,"ahead of climate summit, 2 views from cities i..."
2,19,if you stand outside the old dekalb county cou...,0.9974,"barred from removing confederate monument, cou..."
3,20,you may have spotted pink salt in your superma...,0.4950,pakistan's pink himalayan salt has become a ma...
4,21,"singing in foreign language . the other day , ...",0.9730,"10 years after haiti's earthquake, 'this music..."


In [26]:
# stories.to_csv('stories.csv')

In [27]:
stories.vader.idxmax()

39

In [29]:
stories.lemmatized[39]

'looking ahead to our last day , many of our friend and colleague have been faced with a dilemma. well , how do you say goodbye ? for many of u , finding the right word can be difficult. and when inspiration fails and panic set in , well , we often turn to the long rack of greeting cards. so we ve asked former hallmark greeting card writer and creator of the youtube series greeting card emergency david elli dickerson. he ll join u in a moment. but what s the best goodbye card you ever got ? 800 989 8255. email u talknpr.org. david elli dickerson , with u here in studio 42. nice to have you back on the program . it s a lovely to be back , neal . and i have to say , broadly speaking , there seemed to be two categories. hey , congratulation , you re moving on , or , oh , i m so sorry . right , right. there s the you got fired and , you know , we re not going to see you , or you re going back to school. you re following your dreams. right. yeah . exactly. when you left hallmark , did anybo

In [30]:
stories.title[39]

'so hard to say goodbye: advice for farewell notes'

The first highest-sentiment article is about hallmark greeting cards

In [31]:
stories[stories.vader == 1]

,story_id_num,lemmatized,vader,title
39,620,"looking ahead to our last day , many of our fr...",1.0,so hard to say goodbye: advice for farewell notes
42,623,"this is talk of the nation. i m neal conan , i...",1.0,a look ahead and a farewell to the political j...
43,624,"this is talk of the nation. i m neal conan , i...",1.0,what changes after supreme court rulings on pr...
44,625,"from small country church , to the stage of th...",1.0,gospel legend mavis staples comes 'full circle'
141,898,this is talk of the nation. i m neal conan in ...,1.0,'blood &amp; beauty' breathes new life into th...
...,...,...,...,...
23796,84430,this is talk of the nation. i m neal conan in ...,1.0,state legislatures: agendas and budgets
23995,84850,time now for a story that is decidedly not a n...,1.0,encore: an elf questions santa's naughty or ni...
27366,90916,the political campaign are taking a break. the...,1.0,santa for president in 2016
30850,96419,"now we re going to take a break from news , fr...",1.0,listen: the not-so-true story of santa's naugh...


- Actually...there are 1345 stories with a max vader score

In [32]:
stories[stories.vader == -1]

,story_id_num,lemmatized,vader,title
749,2749,this is talk of the nation. i m neal conan in ...,-1.0,"the epidemiology of gun violence: race, region..."
5731,19881,this is talk of the nation. i m neal conan in ...,-1.0,how do you define a hate crime?
10730,55411,"this is talk of the nation. i m neal conan , i...",-1.0,understanding suicide terrorism and how to sto...
11827,58454,this is talk of the nation. im neal conan in w...,-1.0,deconstructing 'myths about suicide'
15177,66409,this is talk of the nation. i m neal conan in ...,-1.0,the legacy of bush's 'war on terror'
17237,71520,this is talk of the nation. i m neal conan in ...,-1.0,who carries out suicide bombings?
18838,75087,this is talk of the nation. i m neal conan in ...,-1.0,how do we define terrorism?
21189,79688,"this is talk of the nation. i m neal conan , i...",-1.0,should the guantanamo prison be closed?
21287,79858,"from npr news in washington , dc , i m neal co...",-1.0,'war of nerves': a history of chemical weapons


- But only 9 stories with the lowest vader score, including...

In [33]:
stories.lemmatized[749]

'this is talk of the nation. i m neal conan in washington. data from the center for disease control and prevention come to some remarkable conclusion about gun death , race , sex and region. briefly put , white people who die by gunfire are much more likely to be male , rural and suicide black people much more likely to be urban , victim of homicide and again , male . we ll explore the analysis , which wa featured in the washington post. we d also like to hear about your experiences. how ha gun violence affected you , your family and your neighborhood ? 800 989 8255. email u , talknpr.org. you can also join the conversation on our website. that s at npr.org. click on talk of the nation . later in the program , the coast guard and the northwest passage. but first gun death in america. dan keating is data editor and reporter with the washington post and join u now from studio at that newspaper. good to have you on talk of the nation today . good afternoon . and i understand of course , y

In [34]:
stories.title[749]

'the epidemiology of gun violence: race, region and policy'

- A story aboout "gun death , race , sex and region"

In [35]:
darkest_stories = stories[stories.vader == -1]
darkest_stories

,story_id_num,lemmatized,vader,title
749,2749,this is talk of the nation. i m neal conan in ...,-1.0,"the epidemiology of gun violence: race, region..."
5731,19881,this is talk of the nation. i m neal conan in ...,-1.0,how do you define a hate crime?
10730,55411,"this is talk of the nation. i m neal conan , i...",-1.0,understanding suicide terrorism and how to sto...
11827,58454,this is talk of the nation. im neal conan in w...,-1.0,deconstructing 'myths about suicide'
15177,66409,this is talk of the nation. i m neal conan in ...,-1.0,the legacy of bush's 'war on terror'
17237,71520,this is talk of the nation. i m neal conan in ...,-1.0,who carries out suicide bombings?
18838,75087,this is talk of the nation. i m neal conan in ...,-1.0,how do we define terrorism?
21189,79688,"this is talk of the nation. i m neal conan , i...",-1.0,should the guantanamo prison be closed?
21287,79858,"from npr news in washington , dc , i m neal co...",-1.0,'war of nerves': a history of chemical weapons


In [37]:
for story in darkest_stories.title:
    print(story)

the epidemiology of gun violence: race, region and policy
how do you define a hate crime?
understanding suicide terrorism and how to stop it
deconstructing 'myths about suicide'
the legacy of bush's 'war on terror'
who carries out suicide bombings?
how do we define terrorism?
should the guantanamo prison be closed?
'war of nerves': a history of chemical weapons


- Here's a list of the stories with the lowest possible sentiment score